# **文字分類 API（預覽版）**

## **什麼是文字分類？**

顧名思義，文字分類是將標籤或類別應用於文字的過程。

常見用例包括：

- 將電子郵件分類為垃圾郵件或非垃圾郵件
- 根據客戶評論將情緒分析為正面或負面
- 應用標籤來支援工單

## **用機器學習解決文字分類**

分類是機器學習中的一個常見問題。您可以使用多種演算法來訓練分類模型。文字分類是分類的一個子類別別，專門處理原始文字。文字提出了有趣的挑戰，因為您必須考慮文字出現的上下文和語義。因此，編碼含義和上下文可能很困難。近年來，深度學習模型已成為解決自然語言問題的一種有前途的技術。更具體地說，一種稱為轉換器的神經網路已成為解決自然語言問題（如文字分類、翻譯、摘要和問答）的主要方式。

論文中介紹了Transformers [Attention is all you need](https://arxiv.org/abs/1706.03762)。一些用於自然語言任務的流行轉換器架構包括：

- 來自 Transformers (BERT) 的雙向編碼器表示
- 穩健最佳化的 BERT 預訓練方法 (RoBERTa)
- 生成式預訓練 Transformer 2 (GPT-2)
- 生成式預訓練 Transformer 3 (GPT-3)

在高層次上，轉換器是由編碼和解碼層組成的模型架構。編碼器將原始文字作為輸入，並將輸入對映到數字表示（包括上下文）以生成特徵。解碼器使用來自編碼器的訊息來產生輸出，例如在文字分類的情況下的類別或標籤。使這些層如此特別的是注意力的概念。注意力是根據上下文相對於序列中其他輸入的重要性關注輸入的特定部分的想法。例如，假設我正在根據標題對新聞文章進行分類。並非標題中的所有詞都是相關的。在『汽車銷售處於歷史最高水平』這樣的標題中，『銷售』這樣的詞可能會受到更多關注，並導致將文章標記為商業或金融。

像大多數神經網路一樣，從頭開始訓練轉換器可能會很昂貴，因為它們需要大量資料和計算。但是，您不必總是從頭開始訓練。使用稱為微調的技術，您可以採用預先訓練的模型並使用您自己的資料重新訓練特定於您的域或問題的層。這為您提供了一個更適合解決您的問題的模型，而無需從頭開始訓練整個模型的過程。

## **文字分類 API（預覽版）**

現在您已經大致瞭解瞭如何使用深度學習解決文字分類問題，讓我們來看看我們如何將其中的許多技術整合到文字分類 API 中。

文字分類 API 由 [TorchSharp](https://github.com/dotnet/TorchSharp) 提供支援。 TorchSharp 是一個 .NET 函式庫，提供對 libtorch 的訪問，該函式庫為 PyTorch 提供支援。 TorchSharp 包含用於在 .NET 中從頭開始訓練神經網路的建立塊。然而，TorchSharp 元件是底層級別的的，從頭開始建立神經網路具有陡峭的學習曲線。在 ML.NET 中，我們已將部分複雜性抽象到場景級別。

## 安裝套件

要使用文字分類 API，您必須安裝以下軟體套件

- [`Microsoft.ML`](https://www.nuget.org/packages/Microsoft.ML/)
- [`Microsoft.ML.TorchSharp`](https://www.nuget.org/packages/Microsoft.ML.TorchSharp/)
- [`TorchSharp-cpu`](https://www.nuget.org/packages/TorchSharp-cpu/) 如果您使用的是 CPU 或 [`TorchSharp-cuda-windows`](https://www.nuget.org/packages/TorchSharp-cuda-windows/) / [`TorchSharp-cuda-linux`](https://www.nuget.org/packages/TorchSharp-cuda-linux/) 如果您使用的是 GPU .

要啟用 GPU 支援，您還必須安裝 CUDA 依賴項。 有關詳細訊息，請參閱 [GPU 支援指南](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/install-gpu-model-builder#install-dependencies)。

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

#r "nuget:Microsoft.ML,2.0.0-preview.22324.1"
#r "nuget:Microsoft.ML.TorchSharp,0.20.0-preview.22324.1"
#r "nuget:TorchSharp-cpu,0.96.7"
#r "nuget:Microsoft.Data.Analysis,0.20.0-preview.22324.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22324.1 Microsoft.ML, 2.0.0-preview.22324.1 Microsoft.ML.TorchSharp, 0.20.0-preview.22324.1 TorchSharp-cpu, 0.96.7

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

## **新增 using 語句**

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.TorchSharp;

## **初始化 MLContext**

所有 ML.NET 操作都從 MLContext 類開始。 初始化 mlContext 會建立一個新的 ML.NET 環境，該環境可以在模型建立工作流物件之間共享。 它在概念上類似於實體框架中的 DBContext。

In [1]:
var mlContext = new MLContext();

### **下載或查詢資料**
以下程式碼嘗試在幾個已知位置找到資料檔案，或者它將從已知的 GitHub 位置下載它。

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [1]:
var yelp_reviews = EnsureDataSetDownloaded("yelp_labelled.txt");
var columnNames = new [] {"Text", "Sentiment"};
var df = DataFrame.LoadCsvFromString(yelp_reviews, separator:'\t',header:false, columnNames:columnNames);

yelp_labelled.txt found here: C:\dev\csharp-notebooks\machine-learning\data\yelp_labelled.txt


載入資料後，使用 `Head` 方法預覽前三行。

In [1]:
df.Head(3)

index,Text,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0


> 本教程的資料集使用來自 Kotzias 等人的『使用深度特徵從組到單個標籤』的資料集。 KDD 2015，並託管在 UCI 機器學習儲存庫 - Dua, D. 和 Karra Taniskidou, E. (2017)。 [UCI 機器學習儲存庫](http://archive.ics.uci.edu/ml)。 加利福尼亞州歐文：加利福尼亞大學訊息與電腦科學學院。

資料集包含兩列：

- **文字:** 來自 Yelp 的原始評論文字
- **情緒:** 表示評論情緒的二進位值。 0 為負數，1 為正數。

## **將資料拆分為訓練集和測試集**

原始資料集分為兩個子集：訓練和測試。 訓練集是您用來學習資料模式的工具。 測試集用於使用分類任務的評估指標來評估模型的效能。

在這種情況下，80% 的資料用於由『testFraction』引數定義的訓練。 剩下的 20% 用於評估和測試。

In [1]:
var trainTestSplit = mlContext.Data.TrainTestSplit(df, testFraction:0.2);

## **定義訓練通道**

文字分類 API 是多類分類目錄的一部分。 要使用它，請將 `TextClassification` 訓練器新增到您的通道中。

In [1]:
var pipeline =
		mlContext.Transforms.Conversion.MapValueToKey("Label","Sentiment")
			.Append(mlContext.MulticlassClassification.Trainers.TextClassification(sentence1ColumnName: "Text"))
			.Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

## **訓練模型**

使用訓練資料集使用 `Fit` 方法訓練您的模型

In [1]:
var model = pipeline.Fit(trainTestSplit.TrainSet);

## **使用模型進行預測**

透過呼叫『Transform』方法，使用您的模型對測試資料集進行預測。

In [1]:
var predictionIDV = model.Transform(trainTestSplit.TestSet);

呼叫 `Transform` 的結果是帶有您的預測值的 `IDataView`。 為了更容易檢視您的預測，請將 `IDataView` 轉換為 `IDataFrame` 。 在這種情況下，我感興趣的唯一列是 Text、Sentiment（實際值）和 PredictedLabel（預測值）。

In [1]:
var columnsToSelect = new [] {"Text", "Sentiment", "PredictedLabel"};

var predictions = predictionIDV.ToDataFrame(columnsToSelect);

使用 `Tail` 方法預覽預測 `DataFrame` 中的最後三行。

In [1]:
predictions.Tail(3)

index,Text,Sentiment,PredictedLabel
0,"Oh this is such a thing of beauty, this restaurant.",1,0
1,"A greasy, unhealthy meal.",0,1
2,"The best place in Vegas for breakfast (just check out a Sat, or Sun.",1,1


## **評估模型**

您可以使用多種指標來評估模型的效能。 使用 [Evaluate](https://docs.microsoft.com/dotnet/api/microsoft.ml.multiclassclassificationcatalog.evaluate?view=ml-dotnet) 方法使用預測『IDataView』計算模型的評估指標。

In [1]:
var evaluationMetrics = 
	mlContext
		.MulticlassClassification
		.Evaluate(predictionIDV);

然後，顯示評估指標。 有關多類分類評估指標的更多訊息，請參閱 [ML.NET 評估指標指南](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics#evaluation-metrics-for-multi-class- 分類）。

In [1]:
evaluationMetrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
10.53512863047496,-14.199291365827746,0.6737016700983757,0.6737967914438503,0,0,<null>,"[ 10.940300196581468, 10.134267400178105 ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.6777777777777778, 0.6701030927835051 ], PerClassRecall: [ 0.6559139784946236, 0.6914893617021277 ], Counts: [ [ 61, 32 ], [ 29, 65 ] ], NumberOfClasses: 2 }"
